# Impfquoten 7-Tage-Schnitt Top 10 Kantone exportieren
https://app.datawrapper.de/chart/cHMOh/visualize#refine

In [73]:
import pandas as pd
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta

In [75]:
df = pd.read_csv(Path('../bag_covid_19_data_csv/data/COVID19Cases_geoRegion.csv'))

df = df[df.isin(['CH', 'CHFL', 'FL']) == False]
df = df.dropna(axis = 0, subset = ['geoRegion'])

# as Datetime
df['datum'] = pd.to_datetime(df['datum'])

# From yesterday
df = df[df.datum <= (df.datum.max() - timedelta(days=1))]

# Calc per 100'000
df['per100k'] = df['entries'] / df['pop'] * 100000

# Find top 10
df_top = df[df.datum == df.datum.max()].sort_values('entries', ascending=False)

top10 = df_top.head(10)['geoRegion']
df = df[df.geoRegion.isin(top10)]

df_new = []

# Calc rolling for each canton
for g in df.geoRegion.unique():
    df_x = df[df.geoRegion == g].copy()
    df_x['7tageinzidenz'] = round(df_x['per100k'].rolling(window=7).mean(), 1)
    df_new.append(df_x)
    
df_new = pd.concat(df_new)

# Pivot on Canton
df_new = pd.pivot_table(df_new, index='datum', columns='geoRegion', values='7tageinzidenz').reset_index()

df_new.to_clipboard(index=False)
#df_new[['geoRegion', 'datum', 'entries', '7tageinzidenz']].to_clipboard(index=False)